In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
%matplotlib inline 
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 999 

#goal is to make a guess using regression and machine learning of how many rent will be done on the very next hour of a certain hour of a certain day example, prediction of how many bike will be rented on at 13:00 if we have at now 12:00 on a Thurday.

#to achieve this i is necessary to find a correlation between rentals and day of the week, rentals and weather, rentals and time of the day, and rentals and precipitation.

#making a regression with this 4 feature engineering and then do the machine learning make the work with the regression who give us the little failure 

Set A= hour + day + work day or weekend + temperature + precipitation
Set B= number of bikes rented on the next 12 hours. 
Set C = number of bikes rented on the previous 12 hours. 
Set D = number of bikes rented on the prev 7 days. 
Set E = number of bikes rented on the prev 4 weeks.

Feature Set B = capture the next demand on that day. 
Feature Set C = capture the recent demand on that day. 
Feature Set D = capture the recent demand on a certain day of the week. 
Feature Set E = capture the demand on a certain week. 

In [2]:
# Loading dataset 
df = pd.read_csv("boston_2019.csv")
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,2019-01-01 00:09:13.7980,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber
1,2019-01-01 00:33:56.1820,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber
2,2019-01-01 00:41:54.6000,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber
3,2019-01-01 00:43:32.5710,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber
4,2019-01-01 00:49:56.4640,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber


In [3]:
# statistical info 
df.describe()


,start_station_id,end_station_id,bike_id
count,2.522771e+06,2.522771e+06,2.522771e+06
mean,1.423010e+02,1.416275e+02,3.637649e+03
std,1.183249e+02,1.180612e+02,1.287294e+03
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,5.500000e+01,5.400000e+01,2.745000e+03
50%,9.900000e+01,9.800000e+01,3.670000e+03
75%,1.900000e+02,1.900000e+02,4.497000e+03
max,4.460000e+02,4.460000e+02,6.173000e+03


In [4]:
#datatype info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522771 entries, 0 to 2522770
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   start_time          object
 1   end_time            object
 2   start_station_id    int64 
 3   end_station_id      int64 
 4   start_station_name  object
 5   end_station_name    object
 6   bike_id             int64 
 7   user_type           object
dtypes: int64(3), object(5)
memory usage: 154.0+ MB


In [5]:
#unique values 
#to check how much entrance data we have with out counting them twice
df.apply(lambda x: len(x.unique()))

start_time            2522540
end_time              2522546
start_station_id          338
end_station_id            339
start_station_name        363
end_station_name          364
bike_id                  4652
user_type                   2
dtype: int64

##Caution there are more end stations names as start station names, for the problem not necessary to fix it  

In [6]:
#preprocessing data 

In [7]:
#check for null values 
df.isnull().sum()
#if 0 then there are no null values

start_time            0
end_time              0
start_station_id      0
end_station_id        0
start_station_name    0
end_station_name      0
bike_id               0
user_type             0
dtype: int64

#user typer subscriber=1 customer=2

In [8]:
#creating customer or subscriber column as integer value
df['int_user_type']=0
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,int_user_type
0,2019-01-01 00:09:13.7980,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,0
1,2019-01-01 00:33:56.1820,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,0
2,2019-01-01 00:41:54.6000,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,0
3,2019-01-01 00:43:32.5710,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,0
4,2019-01-01 00:49:56.4640,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,0


In [9]:
#defining the transformation from category to numbers for machine learning 
def transform_user_to_int_user(x): 
    if x=='Subscriber': 
        return 1 
    if x=='Customer':
        return 2

#Filling the int_user_type column with nummerical values

In [10]:
df['int_user_type']=df['user_type'].apply(transform_user_to_int_user);
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,int_user_type
0,2019-01-01 00:09:13.7980,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,1
1,2019-01-01 00:33:56.1820,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,1
2,2019-01-01 00:41:54.6000,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,1
3,2019-01-01 00:43:32.5710,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,1
4,2019-01-01 00:49:56.4640,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,1


In [11]:
#checking the values of the columns if int_user_value is integer
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522771 entries, 0 to 2522770
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   start_time          object
 1   end_time            object
 2   start_station_id    int64 
 3   end_station_id      int64 
 4   start_station_name  object
 5   end_station_name    object
 6   bike_id             int64 
 7   user_type           object
 8   int_user_type       int64 
dtypes: int64(4), object(5)
memory usage: 173.2+ MB


#changing dates into days of the week 

In [12]:
#transform from string to datetime object to get the days 
df['start_time']=pd.to_datetime(df['start_time']);

In [13]:
df['start_time']

0         2019-01-01 00:09:13.798
1         2019-01-01 00:33:56.182
2         2019-01-01 00:41:54.600
3         2019-01-01 00:43:32.571
4         2019-01-01 00:49:56.464
                    ...          
2522766   2019-12-31 23:52:10.728
2522767   2019-12-31 23:52:14.927
2522768   2019-12-31 23:54:58.391
2522769   2019-12-31 23:55:54.544
2522770   2019-12-31 23:57:49.672
Name: start_time, Length: 2522771, dtype: datetime64[ns]

In [14]:
df['Day_string']=df['start_time'].dt.day_name()
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,int_user_type,Day_string
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,1,Tuesday
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,1,Tuesday
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,1,Tuesday
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,1,Tuesday
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,1,Tuesday


In [15]:
def transform_day_string_to_int(x):
    if x=='Monday':
        return 1
    if x=='Tuesday':
        return 2
    if x=='Wednesday':
        return 3
    if x=='Thursday':
        return 4
    if x=='Friday':
        return 5
    if x=='Saturday':
        return 6
    if x=='Sunday':
        return 7

In [16]:
df['Day_int']=df['Day_string'].apply(transform_day_string_to_int)
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,int_user_type,Day_string,Day_int
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,1,Tuesday,2
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,1,Tuesday,2
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,1,Tuesday,2
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,1,Tuesday,2
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,1,Tuesday,2


#making hours into integer variables 

In [18]:
df['hour']=df['start_time'].dt.hour
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522771 entries, 0 to 2522770
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_time          datetime64[ns]
 1   end_time            object        
 2   start_station_id    int64         
 3   end_station_id      int64         
 4   start_station_name  object        
 5   end_station_name    object        
 6   bike_id             int64         
 7   user_type           object        
 8   int_user_type       int64         
 9   Day_string          object        
 10  Day_int             int64         
 11  hour                int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 231.0+ MB


#separating working-day=1 from weekend=0 

In [20]:
def workingday_or_weekend(x):
    if x>=1 and x<=5:
        return 1
    if x>=6 and x<=7:
        return 0

In [23]:
df['workingday']=df['Day_int'].apply(workingday_or_weekend);
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,int_user_type,Day_string,Day_int,hour,workingday
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.3360,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,1,Tuesday,2,0,1
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.8800,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,1,Tuesday,2,0,1
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.2730,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,1,Tuesday,2,0,1
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.4260,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,1,Tuesday,2,0,1
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.7010,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,1,Tuesday,2,0,1
